In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from scipy.optimize import minimize
from scipy.optimize import least_squares


In [7]:
from numpy.random import default_rng, SeedSequence

sq = SeedSequence()
seed = sq.entropy        # on sauve la graine pour reproduire les résultats
print('seed = ', seed)
rng = default_rng(sq)
rng.standard_normal(5)

seed =  251591266078601047453343575499368904122


array([ 0.23925415,  0.18966351, -0.28271264,  0.73880656, -2.09208158])

In [8]:
sns.set_style("whitegrid")
mpl.rcParams['figure.dpi'] = 100

params_grid = {"color": 'lightgrey', "linestyle": 'dotted', "linewidth": 0.7 }

In [9]:
# Pamètre du model
N = 10
M = 10000

T = 1
dt = T/N
S0 = 100
K = 100
sigma = 0.1
r = 0.04
m = 3

Comparer le LS avec régression linéaire et le LS lorsqu'on utilise un réseau de neurones (régression non-linéaire) pour apprendre la fonction de continuation. C'est un travail numérique prospectif à faire pour le mardi 19 mars en s'interrogeant sur les différents paramètres: nombres de scénarios, nombre d'epochs, taille de batch, réglage du learning rate, normalisation des données simulées...

Programmer l'algorithme de Longstaff-Schwartz (celui vu en cours, le dernier!) pour un Put bermudéen dans un modèle de Black-Scholes, c'est à dire un put que l'on peut exercer aux dates $k/N$, $k=0,\dots,N$. On prendra $r=0.04$, $\sigma=0.1$, $x_0=100$ et le strike $K=100$ avec $N=10$ dates jusqu'en $T=1$ et le payoff $\phi_k(x) = e^{-r k/N}(K-x)_+$.

Pour cela on va reprendre le code utiliser pour le TP2 :

In [10]:
def brownian_1d(n_times: int, n_paths: int,
                final_time: float=1.0,
                increments: bool=False,
                random_state: np.random.Generator=rng) -> np.array:
    """Simulate paths of standard Brownian motion
    Args:
        n_times: Number of timesteps
        n_paths: Number of paths
        final_time: Final time of simulation
        increments: If `True` the increments of the paths are returned.
        random_state: `np.random.Generator` used for simulation
    Returns:
        `np.array` of shape `(n_times+1, n_paths)` containing the paths if the argument `increments` is `False`
        `np.array` of shape `(n_times, n_paths)` containing the increments if the argument `increments` is `True`
    """
    dB = np.sqrt(final_time / n_times) * random_state.standard_normal((n_times, n_paths))
    if increments:
        return dB
    else:
        brownian = np.zeros((n_times+1, n_paths))
        brownian[1:] = np.cumsum(dB, axis=0)
        return brownian

def black_scholes_1d(n_times: int, n_paths: int,
                     final_time: float=1.0,
                     random_state: np.random.Generator=rng, *,
                     init_value: float,
                     r: float, sigma: float) -> np.array:
    """Simulate paths of Black-Scholes process
    Args:
        n_times: Number of timesteps
        n_paths: Number of paths
        final_time: Final time of simulation
        init_value: `S0`
        r: Interest rate
        sigma: Volatility
        random_state: `np.random.Generator` used for simulation
    Returns:
        `np.array` of shape `(n_times+1, n_paths)` containing the paths
    """
    Bt = brownian_1d(n_times, n_paths)
    times = np.arange(n_times+1)*(1/n_times)
    t = times[:, np.newaxis]
    St = init_value * np.exp((r - 0.5*sigma**2)*t + sigma*Bt)
    return St

Pour trouver la valeur $V_0$, on applique la récurrence backward suivante :

$V_N(x) = \phi_N(x)$

et $V_n(x) = max(\phi_n(x), \Phi(x,\theta_n))$

avec $\theta_n = argmin_\theta \sum_{j=1}^{M} \left( V_{n+1}(X_{n+1}^{(j)}) - \Phi(X_{n}^{(j)}, \theta) \right)^2$

pour $\Phi$ on a choisis des fonctions polynomiales de degrées m :

$\Phi(x) = \theta_0+\theta_1*x+\theta_2*x^2 + ... + \theta_m*x^m$

In [11]:
# psi est ma régression linéaire qui va me permettre d'approcher le résultat théorique
# Le but étant de trouver les theta optimaux
def psi(x, theta, m):
    res = 0
    for i in range(m):
      res += theta[i]*(x**i)
    return res

def phi(x, K, k, N):
    return np.exp(-r*k/N)*np.maximum(K-x, 0)

#meme fonction que la précédente mais elle me permet de manipuler des tableaux
#et sera donc utile lorsque je ferai une régression pour trouver theta à partir d'une simulation de prix Black Scholes
def phi_regression(x, K, k, N):
    return np.exp(-r*k/N)*np.maximum(K-x[k], 0)

#Fonction dont on cherche l'argmin theta
def objective_function(theta, M, S, n, K, thetas):
    result = 0
    for j in range(M):
        result += (V(S[n+1][j], n+1, thetas[-1], N, K) - psi(S[n][j], theta, K))**2
    return result/M

def V(x, n, theta,N, K):
    if n == N:
        return phi(x,K,N,N)
    else:
        return max(phi(x, K,n,N), psi(x, theta, K))

def simulate_V(S,K,r,n,V):
    if n==-1:
        return V
    else :
        V[n] = np.maximum(phi_regression(S,K,n,N),V[n+1])
        return simulate_V(S,K,r,n-1,V)


# Régression Linéaire

In [56]:
def regression_theta(X_t, M, m, n):
    e_m_X = []
    for j in range(M):
        e_m_X_j = [X_t[n][j]**k for k in range(m)]

        e_m_X.append(np.array(e_m_X_j).reshape(-1, 1))
    #print(e_m_X[2])
    A_n_m_M = np.zeros((e_m_X[0].dot(e_m_X[0].T).shape))
    for i in range(M):
        A_n_m_M += e_m_X[j].dot(e_m_X[j].T)
    A_n_m_M = (1/M) * A_n_m_M
    print(A_n_m_M)
    #A_n_m_M = sum(e_m_X[j].dot(e_m_X[j].T) for j in range(M)) / M
    e_m_X = np.array(e_m_X)
    V_simu = np.zeros_like(X_t)
    V_simu[-1] = phi_regression(X_t, K, N, N)
    Z_n_m =  np.array([simulate_V(X_t[:,j],K,r,V_simu.shape[0]-2,V_simu) for j in range(M)])
    print(Z_n_m.shape)
    esperance = [0 for _ in range(m)]
    for j in range(M-1):
        for k in range(m):
            esperance[k] += Z_n_m[n+1][Z_n_m.shape[1]-1][j]* e_m_X[j][k][0]
    esperance = [esperance[k] / M for k in range(m)]

    #print([np.sum(Z_n_m[0][j][n+1] * e_m_X[j]) / M for j in range(M)])
    theta_n_m_M = np.linalg.inv(A_n_m_M+0.00001*np.identity(len(A_n_m_M[0]))).dot(esperance)
    return theta_n_m_M

def longstaff_Schwartz_regression(x0, K, T, N, M, m):
    V_hat = [phi(x0, K,N, N)]
    S_t = black_scholes_1d(n_times=N, n_paths=M, final_time= T, init_value=x0, r = r, sigma = sigma)
    thetas = []#[0 for _ in range(m)]#Peut importe les valeurs, on V_N ne dépend pas de theta
    for n in reversed(range(N)):
        print(n)
        #initial_theta = np.ones(4)

        #theta = minimize(objective_function, initial_theta, args=(M, S_t, n, K, thetas))
        theta = regression_theta(S_t, M, m, n)
        optimized_theta = theta#.x
        print(optimized_theta)
        V_hat.append(max(phi(x0, K,n,N), psi(x0, optimized_theta, m)))
        thetas.append(optimized_theta)
    return V_hat, thetas, S_t

In [57]:
V_hat, thetas, S_t = longstaff_Schwartz_regression(S0, K, T, N, M, 3)
print(V_hat)

9
[[1.00000000e+00 9.75736042e+01 9.52060823e+03]
 [9.75736042e+01 9.52060823e+03 9.28960059e+05]
 [9.52060823e+03 9.28960059e+05 9.06419811e+07]]
(2000, 11, 2000)
[  28299.23900871 1301020.19466677  -13336.70325867]
8
[[1.00000000e+00 9.43123059e+01 8.89481104e+03]
 [9.43123059e+01 8.89481104e+03 8.38890139e+05]
 [8.89481104e+03 8.38890139e+05 7.91176634e+07]]
(2000, 11, 2000)
[  9371.09549043 406053.25664417  -4306.46449459]
7
[[1.00000000e+00 9.08153418e+01 8.24742630e+03]
 [9.08153418e+01 8.24742630e+03 7.48992839e+05]
 [8.24742630e+03 7.48992839e+05 6.80200406e+07]]
(2000, 11, 2000)
[ -11863.70145677 -580233.32582195    6390.59409356]
6
[[1.00000000e+00 9.22769694e+01 8.51503908e+03]
 [9.22769694e+01 8.51503908e+03 7.85742000e+05]
 [8.51503908e+03 7.85742000e+05 7.25058905e+07]]
(2000, 11, 2000)
[  -9162.98935494 -462802.53090896    5016.43972111]
5
[[1.00000000e+00 8.64457533e+01 7.47286826e+03]
 [8.64457533e+01 7.47286826e+03 6.45997726e+05]
 [7.47286826e+03 6.45997726e+05 5.584

In [58]:
print("Le prix d'un Put bermudéen avec régression est ", V_hat[-1], "$")

Le prix d'un Put bermudéen avec régression est  2.20452362548831 $


J'obtiens donc un résultat proche du résultat théorique (environ 2.6$). Cela est sans doute du au fait que ma régression n'est qu'une approximation des formules théoriques mais aussi au fait qu'on approche les espérance par des MC.

## Réseaux de neurones

In [11]:
!pip install torch torchvision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.4 MB/s eta 0:00:00


In [78]:
## Redéfinition des fonctions versions pytorch
def V_pytorch(x, n, models, N, K):
    if n==N:
        return torch.tensor(phi(x,K,n,N))
    else :

        return torch.tensor(np.maximum(phi(x,K,n,N),models[n].forward(torch.tensor(x, dtype=torch.float32).unsqueeze(1))))

In [79]:
import torch
from torch import nn
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

m = 4
class NeuralNetwork(nn.Module):
    def __init__(self,m):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1, m),
            nn.ReLU(),
            nn.Linear(m, m),
            nn.ReLU(),
            nn.Linear(m, m),
            nn.ReLU(),
            nn.Linear(m, 1)
        )



    def forward(self, x):
        out = self.linear_relu_stack(x)
        return out

In [86]:
# Ici on entrainer notre réseau de neurone en utilisant la loss :
def nn_loss(model, S_t, K, n, N, models):
    yy = V_pytorch(S_t[n+1], n+1, models, N, K)
    S_n = torch.tensor(S_t[n], dtype=torch.float32).unsqueeze(1)
    xx = model.forward(S_n)
    loss = ((yy - xx) ** 2).mean()
    return loss

M_nn=2000

def longstaff_Schwartz_nn(x0, K, T, N, M, m, num_epochs):

    S_t = black_scholes_1d(n_times=N, n_paths=M, final_time= T, init_value=S0, r = r, sigma = sigma)
    models = np.array([NeuralNetwork(m) for i in range(N)])
    for i in reversed(range(N)):
        model = NeuralNetwork(m)
        optimizer = optim.SGD(model.parameters(), lr=0.01)
        losses = []
        for epoch in range(num_epochs):
            loss = nn_loss(model, S_t, K, i, N, models)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            print(epoch)
            print(loss)
        for param in model.parameters():
            param.requires_grad_(False)
        models[i] = model
    price = max(phi(S0, K,0,N), models[0].forward(torch.tensor(S0, dtype=torch.float32).unsqueeze(-1)))
    return price

In [89]:
price = longstaff_Schwartz_nn(S0, K, T, N, M_nn, 4, 200)
print(price)

0
tensor(22.7521, dtype=torch.float64, grad_fn=<MeanBackward0>)
1
tensor(22.5147, dtype=torch.float64, grad_fn=<MeanBackward0>)
2
tensor(22.2868, dtype=torch.float64, grad_fn=<MeanBackward0>)
3
tensor(22.0678, dtype=torch.float64, grad_fn=<MeanBackward0>)
4
tensor(21.8574, dtype=torch.float64, grad_fn=<MeanBackward0>)
5
tensor(21.6551, dtype=torch.float64, grad_fn=<MeanBackward0>)
6
tensor(21.4604, dtype=torch.float64, grad_fn=<MeanBackward0>)
7
tensor(21.2730, dtype=torch.float64, grad_fn=<MeanBackward0>)
8
tensor(21.0925, dtype=torch.float64, grad_fn=<MeanBackward0>)
9
tensor(20.9187, dtype=torch.float64, grad_fn=<MeanBackward0>)
10
tensor(20.7511, dtype=torch.float64, grad_fn=<MeanBackward0>)
11
tensor(20.5897, dtype=torch.float64, grad_fn=<MeanBackward0>)
12
tensor(20.4340, dtype=torch.float64, grad_fn=<MeanBackward0>)
13
tensor(20.2840, dtype=torch.float64, grad_fn=<MeanBackward0>)
14
tensor(20.1393, dtype=torch.float64, grad_fn=<MeanBackward0>)
15
tensor(19.9998, dtype=torch.floa

<ipython-input-78-69b186ae4514>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(np.maximum(phi(x,K,n,N),models[n].forward(torch.tensor(x, dtype=torch.float32).unsqueeze(1))))


1
tensor(24.9341, dtype=torch.float64, grad_fn=<MeanBackward0>)
2
tensor(24.3635, dtype=torch.float64, grad_fn=<MeanBackward0>)
3
tensor(23.8204, dtype=torch.float64, grad_fn=<MeanBackward0>)
4
tensor(23.3025, dtype=torch.float64, grad_fn=<MeanBackward0>)
5
tensor(22.8081, dtype=torch.float64, grad_fn=<MeanBackward0>)
6
tensor(22.3355, dtype=torch.float64, grad_fn=<MeanBackward0>)
7
tensor(21.8833, dtype=torch.float64, grad_fn=<MeanBackward0>)
8
tensor(21.4502, dtype=torch.float64, grad_fn=<MeanBackward0>)
9
tensor(21.0352, dtype=torch.float64, grad_fn=<MeanBackward0>)
10
tensor(20.6374, dtype=torch.float64, grad_fn=<MeanBackward0>)
11
tensor(20.2557, dtype=torch.float64, grad_fn=<MeanBackward0>)
12
tensor(19.8895, dtype=torch.float64, grad_fn=<MeanBackward0>)
13
tensor(19.5379, dtype=torch.float64, grad_fn=<MeanBackward0>)
14
tensor(19.2002, dtype=torch.float64, grad_fn=<MeanBackward0>)
15
tensor(18.8759, dtype=torch.float64, grad_fn=<MeanBackward0>)
16
tensor(18.5642, dtype=torch.flo

In [90]:
print("Le prix d'un Put bermudéen avec réseaux de neurones est ", price[0].item(), "$")

Le prix d'un Put bermudéen avec réseaux de neurones est  6.609626293182373 $


Je remarque que en augmentant le nombre d'epochs ou le nombre de path M, le prix semble croitre et est meme supérieur à 2.6

Mon modèle n'est donc pas stable et ne converge pas vers sa valeur théorique.
Il y a donc sans doute une erreur d'implémentation.